# Method to perform Nvidia GPU INT-8 quantization on most transformers model (encoder based)

Quantization is one of the most effective and generic approach to make model inference faster.
Basically, it replaces high precision float numbers in model tensors encoded in 32 or 16 bits by lower precision ones encoded in 8 bits or less:

* it takes less memory
* computation is easier / faster

It can be applied to any model in theory, and, if done well, it should not decrease its accuracy.

The purpose of this notebook is to show 2 processes to perform quantization on most `transformer` architectures.

**TL;DR, we benchmarked Pytorch and Nvidia TensorRT, on both CPU and GPU, with/without quantization, our methods provide the fastest inference by large margin**.

| Framework                  | Precision | Latency (ms) | Accuracy | Speedup    | Hardware |
|:---------------------------|-----------|--------------|----------|:-----------|:--------:|
| Pytorch                    | FP32      | 4000         | 86.8 %   | X 0.02     |   CPU    |
| Pytorch                    | FP16      | 4005         | 86.8 %   | X 0.02     |   CPU    |
| Pytorch                    | **INT-8** | 3670         | 86.8 %   | X 0.02     | **CPU**  |
| Pytorch                    | FP32      | 80           | 86.8 %   | X 1        |   GPU    |
| Pytorch                    | FP16      | 58           | 86.8 %   | X 1.38     |   GPU    |
| ONNX Runtime               | FP32      | 74           | 86.8 %   | X 1.08     |   GPU    |
| ONNX Runtime               | FP16      | 34           | 86.8 %   | X 2.35     |   GPU    |
| ONNX Runtime               | FP32      | 3767         | 86.8 %   | X 0.02     |   CPU    |
| ONNX Runtime               | FP16      | 4607         | 86.8 %   | X 0.02     |   CPU    |
| ONNX Runtime               | **INT-8** | 3712         | 86.8 %   | X 0.02     | **CPU**  |
| TensorRT                   | FP16      | 30           | 86.8 %   | X 2.67     |   GPU    |
| TensorRT (**our method 1**)| **INT-8** | 15           | 84.4 %   | **X 5.33** | **GPU**  |
| TensorRT (**our method 2**)| **INT-8** | 16           | 85.8 %   | **X 5.00** | **GPU**  |

> measures done on a Nvidia RTX 3090 GPU + 12 cores i7 Intel CPU (support AVX-2 instructions)
>
> architecture `Roberta-base` with batch of size 32 / seq len 256, similar results obtained for other sizes/seq len not included in the table.
>
> accuracy obtained after a single epoch, no LR search or any hyper parameter optimization
>
> CPU measures are a bit unfair, it's still possible to push performance a bit by adding lots of (Python related) complexities and using last generation CPU, still those measurements are indicative of orders of magnitude to expect from Pytorch+CPU deployment.
>
> same kind of acceleration is observed on all seq len / batch sizes


## A (very) short intro to INT-8 quantization

Basic idea behind model quantization is to replace tensors made of float numbers (usually encoded on 32 bits) by lower precision representation (integers encoded on 8 bits for Nvidia GPUs).
Therefore computation is faster and model memory footprint is lower. Making tensor storage smaller makes memory transfer faster... and is also a source of computation acceleration.
This technic is very interesting for its trade-off: you reduce inference time significantly, and when dataset is large enough, it costs close to nothing in accuracy.

Replacing float numbers by integers is done through a mapping.
This step is called `calibration`, and its purpose is to compute for each tensor or each channel of a tensor (one of its dimensions) a range of all possible values and then define a scale and a distribution center to map float numbers to 8 bits integers.
The process is well described in this [Nvidia presentation](https://on-demand.gputechconf.com/gtc/2017/presentation/s7310-8-bit-inference-with-tensorrt.pdf).

There are several ways to perform quantization, depending of how and when the `calibration` is performed:

* dynamically: the mapping is done during the inference, there are some overhead but it's easy to put in place and usually the accuracy is preserved,
* statically, after training (`post training quantization` or `PTQ`): this way is efficient, but it may have a significant accuracy cost,
* statically, before training (`quantization aware training` or `QAT`): this way is efficient and has a low accuracy cost as the weights will take care of the result

In this guide we will focus on the third option: `QAT`.

During the quantization aware *training*:

* in the inside, Pytorch will train with high precision float numbers,
* on the outside, Pytorch will simulate that a quantization has already been applied and output results accordingly (for loss computation for instance)

The simulation process is done through the add of quantization / dequantization nodes, most often called `QDQ`, it's an abbreviation you will see often in quantization world.

You can check this [high quality blog post](https://leimao.github.io/article/Neural-Networks-Quantization/) for more information.

## Why this notebook?

CPU quantization is supported out of the box by `Pytorch` and `ONNX Runtime`.
**GPU quantization on the other side requires specific tools and process to be applied**.

In the specific case of `transformer` models, until recently (december 2021), the only way shown by Nvidia is to build manually the graph of our models in `TensorRT`. This is a low level approach, based on GPU capacity knowledge (which operators are supported, etc.). It's certainly out of reach of most NLP practitioners and is very time consuming to update/adapt to new architectures.

Hopefully, Nvidia added to Hugging Face `transformer` library a new model called `QDQBert` few weeks ago.
Basically, it's a vanilla `Bert` architecture which supports INT-8 quantization.
It doesn't support any other architecture out of the box, like `Albert`, `Roberta`, or `Electra`.
Nvidia also provide a demo dedicated to the SQuaD task.

This open the door to extension of the approach to other architectures.

To be both simple and cover most use cases, in this notebook we will see:

* how to perform GPU quantization on **any** transformer model (not just Bert) using a simple trick, a `transplatation`
* how to perform GPU quantization on `QDQRoberta`, a custom model similar to `QDQBert` and supported by `transformer-deploy` library
* how to apply quantization to a common task like classification (which is easier to understand than question answering)
* measure performance gain (latency)


## Project setup

### Dependencies installation

We install `master` branch of `transfomers` library to use a new model: **QDQBert** and `transformer-deploy` to leverage `TensorRT` models (TensorRT API is not something simple to master, it's highly advised to use a wrapper). Your machine should have Nvidia CUDA 11.X, TensorRT 8.2.1 and cuBLAS installed. It's said to be tricky to install, in my experience, just follow Nvidia instructions **and nothing else**, it should work out of the box. Docker image with TensorRT 8.2.1 has not yet been released, this notebook will be updated when it's ready.

In [ ]:
#! pip install git+https://github.com/huggingface/transformers
#! pip install git+https://github.com/ELS-RD/transformer-deploy
#! pip install sklearn datasets
#! pip install pytorch-quantization --extra-index-url https://pypi.ngc.nvidia.com
# or install pytorch-quantization from https://github.com/NVIDIA/TensorRT/tree/master/tools/pytorch-quantization

Check the GPU is enabled and usable.

In [ ]:
! nvidia-smi

Thu Dec  9 19:47:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:03:00.0  On |                  N/A |
| 67%   55C    P8    45W / 350W |    286MiB / 24267MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
import transformers
import datasets
from typing import OrderedDict as OD, List, Dict, Union
import torch
from torch import Tensor
from transformers import (
    AutoModelForSequenceClassification,
    PreTrainedModel,
    QDQBertForSequenceClassification,
    BertForSequenceClassification,
    TrainingArguments,
    Trainer,
    IntervalStrategy,
    AutoTokenizer,
    PreTrainedTokenizer,
)
from datasets import load_dataset, load_metric
from transformer_deploy.QDQModels.QDQRoberta import QDQRobertaForSequenceClassification
import pytorch_quantization.nn as quant_nn
from pytorch_quantization.tensor_quant import QuantDescriptor
from pytorch_quantization import calib
import logging
from datasets import DatasetDict
from transformer_deploy.backends.trt_utils import build_engine, get_binding_idxs, infer_tensorrt, load_engine
from transformer_deploy.backends.ort_utils import convert_to_onnx
from collections import OrderedDict
from transformer_deploy.benchmarks.utils import track_infer_time, print_timings
from pycuda._driver import Stream
import tensorrt as trt
from tensorrt.tensorrt import IExecutionContext, Logger, Runtime
import pycuda.autoinit

Set logging to `error` to make the `notebook` more readable on Github.

In [ ]:
log_level = logging.ERROR
logging.getLogger().setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()
trt_logger: Logger = trt.Logger(trt.Logger.ERROR)
transformers.logging.set_verbosity_error()

### Download data

This part is inspired from an [official Notebooks from Hugging Face](https://github.com/huggingface/notebooks/blob/master/examples/text_classification.ipynb).

In [ ]:
task = "mnli"
num_labels = 3
model_checkpoint = "roberta-base"
batch_size = 32
max_seq_len = 256
validation_key = "validation_matched"
timings: Dict[str, List[float]] = dict()

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark).

In [ ]:
dataset = load_dataset("glue", task)
metric = load_metric("glue", task)
dataset

  0%|          | 0/5 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

### Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

In [ ]:
tokenizer: PreTrainedTokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We can them write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True` and `padding="max_length"`. This will ensure that all sequences have the same size.

In [ ]:
def preprocess_function(examples):
    return tokenizer(
        examples["premise"], examples["hypothesis"], truncation=True, padding="max_length", max_length=max_seq_len
    )

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Some functions required for training and exporting the model:

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)


def calibrate(model: PreTrainedModel, encoded_dataset: DatasetDict, nb_sample: int = 128) -> PreTrainedModel:
    # Find the TensorQuantizer and enable calibration
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                module.disable_quant()
                module.enable_calib()
            else:
                module.disable()

    with torch.no_grad():
        for start_index in tqdm(range(0, nb_sample, batch_size)):
            end_index = start_index + batch_size
            data = encoded_dataset["train"][start_index:end_index]
            input_torch = {
                k: torch.tensor(v, dtype=torch.long, device="cpu")
                for k, v in data.items()
                if k in ["input_ids", "attention_mask", "token_type_ids"]
            }
            model(**input_torch)

    # Finalize calibration
    for name, module in model.named_modules():
        if isinstance(module, quant_nn.TensorQuantizer):
            if module._calibrator is not None:
                if isinstance(module._calibrator, calib.MaxCalibrator):
                    module.load_calib_amax()
                else:
                    module.load_calib_amax("percentile", percentile=99.99)
                module.enable_quant()
                module.disable_calib()
            else:
                module.enable()

    model.cuda()
    return model


def convert_tensor(data: OD[str, List[List[int]]], output: str) -> OD[str, Union[np.ndarray, torch.Tensor]]:
    input: OD[str, Union[np.ndarray, torch.Tensor]] = OrderedDict()
    for k in ["input_ids", "attention_mask", "token_type_ids"]:
        if k in data:
            v = data[k]
            if output == "torch":
                value = torch.tensor(v, dtype=torch.long, device="cuda")
            elif output == "np":
                value = np.asarray(v, dtype=np.int32)
            else:
                raise Exception(f"unknown output type: {output}")
            input[k] = value
    return input

Some `TensorRT` reused variables:

In [ ]:
runtime: Runtime = trt.Runtime(trt_logger)
profile_index = 0

Measure accuracy for ONNX Runtime and TensorRT:

In [ ]:
validation_labels = [item["label"] for item in encoded_dataset[validation_key]]


def measure_accuracy(infer, int64: bool) -> float:
    outputs = list()
    for start_index in tqdm(range(0, len(encoded_dataset[validation_key]), batch_size)):
        end_index = start_index + batch_size
        data = encoded_dataset[validation_key][start_index:end_index]
        inputs: OD[str, np.ndarray] = convert_tensor(data=data, output="np")
        if int64:
            for k, v in inputs.items():
                inputs[k] = v.astype(np.int64)
        output = infer(inputs)
        output = np.argmax(output[0], axis=1).astype(int).tolist()
        outputs.extend(output)
    return np.mean(np.array(outputs) == np.array(validation_labels))

## Fine-tuning model

Now that our data are ready, we can download the pretrained model and fine-tune it.

Default parameters to be used for the training:

In [ ]:
nb_step = 1000
strategy = IntervalStrategy.STEPS
args = TrainingArguments(
    f"{model_checkpoint}-{task}",
    evaluation_strategy=strategy,
    eval_steps=nb_step,
    logging_steps=nb_step,
    save_steps=nb_step,
    save_strategy=strategy,
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size * 2,
    num_train_epochs=1,
    fp16=True,
    group_by_length=True,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to=[],
)

## Method 1: `Transplantation` of weights from a source model to an optimized architecture

Transplantation idea is to export weights from one model and use them in another one.
In our case, the source are `Roberta` weights and the target is `Bert` archtecture which is highly optimized on `TensorRT` for GPU quantization.

Indeed, not all models are quantization compliant. The optimization engine (`TensorRT`) search for some patterns and will fail to opimize the model if it doesn't find them. It requires the Pytorch code to be written in a certain way and use certain operations. For that reason, it's a good idea to reuse an architecture highly optimized.

We will leverage the fact that since `Bert` have been released, very few improvements have been brought to the transformer architecture (at least for encoder only models).
Better models appeared, and most of the work has been done to improve the pretraining step (aka the weights).
So the idea will be to take the weights from those new models and put them inside `Bert` architecture.

The process described below should work for most architectures.

**steps**:

* load `Bert` model
* retrieve layer/weight names
* load target model (here `Roberta`)
* replace weight/layer names with those from `Roberta`
* override the architecture name in model configuration

If there is no 1 to 1 correspondance (it happens), try to keep at least token embeddings and self attention. Of course, it's possible that if a model is very different, the transplant may cost some accuracy. In our experience, if your trainset is big enough it should not happen.


In [ ]:
model_bert: PreTrainedModel = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=num_labels
)
bert_keys = list(model_bert.state_dict().keys())
del model_bert

model_roberta: PreTrainedModel = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=num_labels
)
model_roberta.save_pretrained("roberta-in-bert")
del model_roberta
model_weights: OD[str, Tensor] = torch.load("roberta-in-bert/pytorch_model.bin")

# Roberta -> Bert, there is 1 to 1 correspondance, for other models, you may need to create your own mapping.
for bert_key in bert_keys:
    # pop remove the first weights from the Ordered dict ...
    _, weight = model_weights.popitem(last=False)
    # ... and we re-insert them, in order, with a new key
    model_weights[bert_key] = weight

# we re-export the weights
torch.save(model_weights, "roberta-in-bert/pytorch_model.bin")
del model_weights

We override the architecture name to make `transformers` believe it is `Bert`...

In [ ]:
# =====> change architecture to bert base <======
import json

with open("roberta-in-bert/config.json") as f:
    content = json.load(f)
    content["architectures"] = ["bert"]

with open("roberta-in-bert/config.json", mode="w") as f:
    json.dump(content, f)

## Model training

The goal of this first training is to update weights to the new architecture to help the next step, the calibration.
Indeed, `Roberta` architecture is a bit different from vanilla `Bert`, for instance position embeddings are not managed the same way, as they are at the very bottom of the model, they impact all model layers.
If we skip this step, the value ranges computed during the calibration step may be very wrong and the `QAT` would provide low accuracy score.

In [ ]:
transformers.logging.set_verbosity_error()
model_bert = BertForSequenceClassification.from_pretrained("roberta-in-bert", num_labels=num_labels)
model_bert = model_bert.cuda()

trainer = Trainer(
    model_bert,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
transformers.logging.set_verbosity_error()
trainer.train()
print(trainer.evaluate())
model_bert.save_pretrained("roberta-in-bert-trained")
del trainer
del model_bert

[INFO|trainer.py:437] 2021-12-09 19:48:11,412 >> Using amp half precision backend


{'loss': 0.7474, 'learning_rate': 9.1875814863103e-06, 'epoch': 0.08}
{'eval_loss': 0.5083153247833252, 'eval_accuracy': 0.8007131940906775, 'eval_runtime': 18.9412, 'eval_samples_per_second': 518.182, 'eval_steps_per_second': 8.13, 'epoch': 0.08}
{'loss': 0.5457, 'learning_rate': 8.372718383311604e-06, 'epoch': 0.16}
{'eval_loss': 0.47982481122016907, 'eval_accuracy': 0.8163015792154865, 'eval_runtime': 19.417, 'eval_samples_per_second': 505.486, 'eval_steps_per_second': 7.931, 'epoch': 0.16}
{'loss': 0.5075, 'learning_rate': 7.557855280312908e-06, 'epoch': 0.24}
{'eval_loss': 0.4634084105491638, 'eval_accuracy': 0.8218033622007132, 'eval_runtime': 19.6824, 'eval_samples_per_second': 498.67, 'eval_steps_per_second': 7.824, 'epoch': 0.24}
{'loss': 0.483, 'learning_rate': 6.743807040417211e-06, 'epoch': 0.33}
{'eval_loss': 0.42370399832725525, 'eval_accuracy': 0.8344370860927153, 'eval_runtime': 19.5405, 'eval_samples_per_second': 502.29, 'eval_steps_per_second': 7.881, 'epoch': 0.33}
{

## Quantization

Below we will start the quantization process.
It follow those steps:

* perform the calibration
* perform a quantization aware training

By passing validation values to the model, we will calibrate it, meaning it will get the right range / scale to convert FP32 weights to int-8 ones.

### Calibration

#### Activate histogram calibration

There are several kinds of calbrators, below we use the percentile one (99.99p) (`histogram`), basically, its purpose is to just remove the most extreme values before computing range / scale.
The other option in NLP is `max`, it's much faster but expect lower accuracy.

Second calibration option, choose between calibration done at the tensor level or per channel (finer grained value ranges, a bit slower).
Calibration is based on few samples (in our case 128 sequences).

In [ ]:
# you can also use "max" instead of "historgram"
input_desc = QuantDescriptor(num_bits=8, calib_method="histogram")
# below we do per-channel quantization for weights, set axis to None to get a per tensor calibration
weight_desc = QuantDescriptor(num_bits=8, axis=(0,))
quant_nn.QuantLinear.set_default_quant_desc_input(input_desc)
quant_nn.QuantLinear.set_default_quant_desc_weight(weight_desc)

#### Perform calibration

During this step we will enable the calibration nodes, and pass some representative data to the model.
It will then be used to compute the scale/range.

Official recommendations from Nvidia is to calibrate over thousands of examples from the validation set.
Here we use 128 examples because it's a slow process. It's enough to be close from the original accuracy.

In [ ]:
# keep it on CPU
model_q = QDQBertForSequenceClassification.from_pretrained("roberta-in-bert-trained", num_labels=num_labels)
model_q = calibrate(model=model_q, encoded_dataset=encoded_dataset)
model_q.save_pretrained("roberta-in-bert-trained-quantized")

  0%|          | 0/4 [00:00<?, ?it/s]

### Quantization Aware Training (QAT)

The query aware training is not a mandatory step, but **highly** recommended to get the best accuracy. Basically we will redo the training with the quantization enabled and a low learning rate to avoid overfitting.

In [ ]:
model_q = QDQBertForSequenceClassification.from_pretrained("roberta-in-bert-trained-quantized", num_labels=num_labels)
model_q = model_q.cuda()

args.learning_rate = 1e-6

trainer = Trainer(
    model_q,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
transformers.logging.set_verbosity_error()
print(trainer.evaluate())
trainer.train()
print(trainer.evaluate())
model_q.save_pretrained("roberta-in-bert-trained-quantized-bis")
del model_q
del trainer

[INFO|trainer.py:437] 2021-12-09 20:39:35,717 >> Using amp half precision backend


{'eval_loss': 0.4307818114757538, 'eval_accuracy': 0.8327050433010698, 'eval_runtime': 45.8394, 'eval_samples_per_second': 214.117, 'eval_steps_per_second': 3.36}
{'eval_loss': 0.4307818114757538, 'eval_accuracy': 0.8327050433010698, 'eval_runtime': 45.8394, 'eval_samples_per_second': 214.117, 'eval_steps_per_second': 3.36}
{'loss': 0.4542, 'learning_rate': 9.187581486310299e-07, 'epoch': 0.08}
{'eval_loss': 0.4258573651313782, 'eval_accuracy': 0.8376974019358125, 'eval_runtime': 46.5114, 'eval_samples_per_second': 211.023, 'eval_steps_per_second': 3.311, 'epoch': 0.08}
{'loss': 0.4422, 'learning_rate': 8.372718383311604e-07, 'epoch': 0.16}
{'eval_loss': 0.4214017987251282, 'eval_accuracy': 0.8395313295975547, 'eval_runtime': 46.5334, 'eval_samples_per_second': 210.924, 'eval_steps_per_second': 3.309, 'epoch': 0.16}
{'loss': 0.4268, 'learning_rate': 7.557855280312907e-07, 'epoch': 0.24}
{'eval_loss': 0.41808152198791504, 'eval_accuracy': 0.8423841059602649, 'eval_runtime': 46.5578, 'ev

### Benchmark

#### Export a `QDQ Pytorch` model on `ONNX`, we need to enable fake quantization mode from Pytorch.

In [ ]:
data = encoded_dataset["train"][0:3]
input_torch = convert_tensor(data, output="torch")

model_q = QDQBertForSequenceClassification.from_pretrained(
    "roberta-in-bert-trained-quantized-bis", num_labels=num_labels
)
model_q = model_q.cuda()
from pytorch_quantization.nn import TensorQuantizer

TensorQuantizer.use_fb_fake_quant = True
convert_to_onnx(model_q, output_path="model_q.onnx", inputs_pytorch=input_torch, opset=13)
TensorQuantizer.use_fb_fake_quant = False
# del model_q

/home/geantvert/.local/share/virtualenvs/fast_transformer/lib/python3.9/site-packages/pytorch_quantization/nn/modules/tensor_quantizer.py:285: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  inputs, amax.item() / bound, 0,
/home/geantvert/.local/share/virtualenvs/fast_transformer/lib/python3.9/site-packages/pytorch_quantization/nn/modules/tensor_quantizer.py:291: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  quant_dim = list(amax.shape).index(list(amax_sequeeze.shape)[0])


#### Convert `ONNX` graph to `TensorRT` engine

In [ ]:
engine = build_engine(
    runtime=runtime,
    onnx_file_path="model_q.onnx",
    logger=trt_logger,
    min_shape=(1, max_seq_len),  # 1 in batch size to support batch from size 1 to 32
    optimal_shape=(batch_size, max_seq_len),
    max_shape=(batch_size, max_seq_len),
    workspace_size=10000 * 1024 * 1024,
    fp16=False,
    int8=True,
)

In [ ]:
# same thing from command line
# !/usr/src/tensorrt/bin/trtexec --onnx=model_q.onnx --shapes=input_ids:32x256,attention_mask:32x256 --int8 --workspace=10000 --saveEngine="test.plan"

#### Prepare input and output buffer

In [ ]:
stream: Stream = pycuda.driver.Stream()
context: IExecutionContext = engine.create_execution_context()
context.set_optimization_profile_async(profile_index=profile_index, stream_handle=stream.handle)
input_binding_idxs, output_binding_idxs = get_binding_idxs(engine, profile_index)  # type: List[int], List[int]

In [ ]:
data = encoded_dataset["train"][0:batch_size]
input_np: Dict[str, np.ndarray] = convert_tensor(data, output="np")

#### Inference on `TensorRT`

We first check that inference is working correctly:

In [ ]:
tensorrt_output = infer_tensorrt(
    context=context,
    host_inputs=input_np,
    input_binding_idxs=input_binding_idxs,
    output_binding_idxs=output_binding_idxs,
    stream=stream,
)
print(tensorrt_output)

[array([[ 0.74428475,  0.92069125, -1.8263749 ],
       [ 1.851499  , -1.0653014 , -1.4172065 ],
       [ 1.7828548 , -1.1306885 , -1.1852558 ],
       [ 1.6740881 , -0.7420906 , -1.5647126 ],
       [ 2.2817519 ,  0.01615529, -2.7685544 ],
       [ 3.1013348 , -0.48788828, -3.3121827 ],
       [-3.0679533 ,  2.708288  ,  0.70968   ],
       [ 3.1545656 , -1.0913979 , -2.6073706 ],
       [-0.3026344 , -1.7703965 ,  1.6011946 ],
       [-3.2131557 , -0.5275665 ,  3.786335  ],
       [ 2.2266033 , -1.2310914 , -1.523544  ],
       [-1.5110059 , -0.46988845,  1.7940781 ],
       [-2.4409676 ,  3.7142613 , -0.73455316],
       [-1.8158143 ,  1.9259161 , -0.05558195],
       [-0.33427513, -0.48280472,  0.6140785 ],
       [ 2.3686104 , -1.4665173 , -1.5184819 ],
       [ 3.58267   , -1.1251179 , -3.060151  ],
       [-2.4983776 , -2.0526152 ,  4.5359097 ],
       [-3.441052  , -0.6358736 ,  4.1798487 ],
       [-2.2326443 ,  4.032728  , -1.1005057 ],
       [ 3.4742196 , -0.98982847, -3.24

Measure of the accuracy when `TensorRT` is the engine:

In [ ]:
infer_trt = lambda inputs: infer_tensorrt(
    context=context,
    host_inputs=inputs,
    input_binding_idxs=input_binding_idxs,
    output_binding_idxs=output_binding_idxs,
    stream=stream,
)

measure_accuracy(infer=infer_trt, int64=False)

  0%|          | 0/307 [00:00<?, ?it/s]

0.8444218033622007

Latency measures:

In [ ]:
time_buffer = list()
for _ in range(100):
    with track_infer_time(time_buffer):
        _ = infer_tensorrt(
            context=context,
            host_inputs=input_np,
            input_binding_idxs=input_binding_idxs,
            output_binding_idxs=output_binding_idxs,
            stream=stream,
        )

print_timings(name="TensorRT (INT-8)", timings=time_buffer)
del engine, context  # delete all tensorrt objects

[TensorRT (INT-8)] mean=14.63ms, sd=0.33ms, min=13.94ms, max=17.54ms, median=14.58ms, 95p=14.76ms, 99p=15.38ms


## Method 2: use a dedicated QDQ model

In method 2, the idea is to take the source code of a specific model and add manually in the source code `QDQ` nodes. That way, quantization will work out of the box for this architecture.
We have started with `QDQRoberta` a quantization compliant `Roberta` model.

To adapt to another architecture, one need to:

* replace linear layers with their quantized version
* replace operations not supported out of the box by `TensorRT` by a similar code supporting the operation.

> concrete examples on `Roberta` architecture: in HF library, there is a `cumsum` in the position embedding generation. Something very simple. It takes as input an integer tensor and output an integer tensor. It happens that the `cumsum` operator from TensorRT supports float but not integer (https://github.com/onnx/onnx-tensorrt/blob/master/docs/operators.md). It leads to a crash during the model conversion with a strange error message. Converting the input to float tensor fix the issue. Not complex, but requires some knowledge.

The process below is a bit simpler than the method 1:

* Calibrate
* Quantization Aware training (QAT)

> there are many ways to get a QDQ model, you can modify Pytorch source code like here, patch ONNX graph (this approach is used at Microsoft for instance) or leverage the new FX Pytorch interface. Modifying the source code is the most straight forward so we choosed to do it that way.


### Calibration

In [ ]:
input_desc = QuantDescriptor(num_bits=8, calib_method="histogram")
# below we do per-channel quantization for weights, set axis to None to get a per tensor calibration
weight_desc = QuantDescriptor(num_bits=8, axis=(0,))
quant_nn.QuantLinear.set_default_quant_desc_input(input_desc)
quant_nn.QuantLinear.set_default_quant_desc_weight(weight_desc)

# keep it on CPU
model_roberta: PreTrainedModel = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=num_labels
)
model_roberta.save_pretrained("roberta-untrained-quantized")
del model_roberta

model_roberta_q: PreTrainedModel = QDQRobertaForSequenceClassification.from_pretrained("roberta-untrained-quantized")
model_roberta_q = calibrate(model=model_roberta_q, encoded_dataset=encoded_dataset)
model_roberta_q.save_pretrained("roberta-untrained-quantized")
del model_roberta_q

  0%|          | 0/4 [00:00<?, ?it/s]

### Quantization Aware Training (QAT)

In [ ]:
model_roberta_q: PreTrainedModel = QDQRobertaForSequenceClassification.from_pretrained(
    "roberta-untrained-quantized", num_labels=num_labels
)
model_roberta_q = model_roberta_q.cuda()

args.learning_rate = 1e-5

trainer = Trainer(
    model_roberta_q,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
transformers.logging.set_verbosity_error()
trainer.train()
print(trainer.evaluate())
model_roberta_q.save_pretrained("roberta-trained-quantized")
del model_roberta_q

[INFO|trainer.py:437] 2021-12-09 22:10:02,417 >> Using amp half precision backend


{'loss': 0.7446, 'learning_rate': 9.1875814863103e-06, 'epoch': 0.08}
{'eval_loss': 0.5072791576385498, 'eval_accuracy': 0.8059093224656139, 'eval_runtime': 47.7062, 'eval_samples_per_second': 205.738, 'eval_steps_per_second': 3.228, 'epoch': 0.08}
{'loss': 0.5424, 'learning_rate': 8.372718383311604e-06, 'epoch': 0.16}
{'eval_loss': 0.4495479464530945, 'eval_accuracy': 0.8264900662251655, 'eval_runtime': 46.5832, 'eval_samples_per_second': 210.698, 'eval_steps_per_second': 3.306, 'epoch': 0.16}
{'loss': 0.5071, 'learning_rate': 7.558670143415907e-06, 'epoch': 0.24}
{'eval_loss': 0.4541535973548889, 'eval_accuracy': 0.8242485990830362, 'eval_runtime': 50.0162, 'eval_samples_per_second': 196.236, 'eval_steps_per_second': 3.079, 'epoch': 0.24}
{'loss': 0.4854, 'learning_rate': 6.743807040417211e-06, 'epoch': 0.33}
{'eval_loss': 0.4110897183418274, 'eval_accuracy': 0.8425878757004585, 'eval_runtime': 47.5433, 'eval_samples_per_second': 206.443, 'eval_steps_per_second': 3.239, 'epoch': 0.33

### Benchmark

#### Export a `QDQ Pytorch` model on `ONNX`, we need to enable fake quantization mode from Pytorch.

In [ ]:
model_roberta_q: PreTrainedModel = QDQRobertaForSequenceClassification.from_pretrained(
    "roberta-trained-quantized", num_labels=num_labels
)
model_roberta_q = model_roberta_q.cuda()

data = encoded_dataset["train"][1:3]
input_torch = convert_tensor(data, output="torch")

from pytorch_quantization.nn import TensorQuantizer

TensorQuantizer.use_fb_fake_quant = True
convert_to_onnx(model_pytorch=model_roberta_q, output_path="roberta_q.onnx", inputs_pytorch=input_torch, opset=13)
TensorQuantizer.use_fb_fake_quant = False

/home/geantvert/.local/share/virtualenvs/fast_transformer/lib/python3.9/site-packages/pytorch_quantization/nn/modules/tensor_quantizer.py:285: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  inputs, amax.item() / bound, 0,
/home/geantvert/.local/share/virtualenvs/fast_transformer/lib/python3.9/site-packages/pytorch_quantization/nn/modules/tensor_quantizer.py:291: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  quant_dim = list(amax.shape).index(list(amax_sequeeze.shape)[0])


#### Convert `ONNX` graph to `TensorRT` engine

In [ ]:
engine = build_engine(
    runtime=runtime,
    onnx_file_path="roberta_q.onnx",
    logger=trt_logger,
    min_shape=(1, max_seq_len),
    optimal_shape=(batch_size, max_seq_len),
    max_shape=(batch_size, max_seq_len),
    workspace_size=10000 * 1024 * 1024,
    fp16=False,
    int8=True,
)

In [ ]:
# same conversion from the terminal
#!/usr/src/tensorrt/bin/trtexec --onnx=roberta_q.onnx --shapes=input_ids:32x256,attention_mask:32x256 --int8 --workspace=10000 --saveEngine="test.plan"

#### Prepare input and output buffer

In [ ]:
stream: Stream = pycuda.driver.Stream()
context: IExecutionContext = engine.create_execution_context()
context.set_optimization_profile_async(profile_index=profile_index, stream_handle=stream.handle)
input_binding_idxs, output_binding_idxs = get_binding_idxs(engine, profile_index)  # type: List[int], List[int]

In [ ]:
data = encoded_dataset["train"][0:batch_size]
input_torch: OD[str, torch.Tensor] = convert_tensor(data=data, output="torch")
input_np: OD[str, np.ndarray] = convert_tensor(data=data, output="np")

#### Inference on `TensorRT`

We first check that inference is working correctly:

In [ ]:
tensorrt_output = infer_tensorrt(
    context=context,
    host_inputs=input_np,
    input_binding_idxs=input_binding_idxs,
    output_binding_idxs=output_binding_idxs,
    stream=stream,
)
print(tensorrt_output)

[array([[-0.51485693,  1.7172506 , -1.5262733 ],
       [ 2.353083  , -1.1611496 , -1.5365003 ],
       [ 1.7413325 , -0.5790743 , -1.659783  ],
       [ 1.564936  , -0.49288243, -1.3157034 ],
       [ 1.625774  , -0.06960616, -2.3415694 ],
       [ 3.4986691 , -1.4058641 , -2.909094  ],
       [-2.8577392 ,  2.3696911 ,  0.9519228 ],
       [ 3.3248267 , -1.3577703 , -2.7853382 ],
       [ 0.24115235, -1.2206222 ,  1.9764783 ],
       [-2.1684752 , -0.5929435 ,  4.1980004 ],
       [ 2.7209766 , -0.85320175, -2.54238   ],
       [-1.4474616 , -0.5539231 ,  3.543574  ],
       [-2.4900246 ,  2.5807233 ,  0.29105982],
       [-2.5218582 ,  2.4110076 ,  0.4147416 ],
       [-0.17686448,  0.19154471,  0.5593225 ],
       [ 2.7820387 , -0.92807496, -2.466081  ],
       [ 3.279974  , -1.1566027 , -3.082859  ],
       [-2.0141928 , -1.5038209 ,  4.759576  ],
       [-2.8134325 , -0.09846646,  4.3754187 ],
       [-2.4098513 ,  3.2899659 , -1.0759622 ],
       [ 3.3163776 , -1.0768431 , -3.22

Measure of the accuracy:

In [ ]:
infer_trt = lambda inputs: infer_tensorrt(
    context=context,
    host_inputs=inputs,
    input_binding_idxs=input_binding_idxs,
    output_binding_idxs=output_binding_idxs,
    stream=stream,
)

measure_accuracy(infer=infer_trt, int64=False)

  0%|          | 0/307 [00:00<?, ?it/s]

0.8577687213448802

Latency measures:

In [ ]:
time_buffer = list()
for _ in range(100):
    with track_infer_time(time_buffer):
        _ = infer_tensorrt(
            context=context,
            host_inputs=input_np,
            input_binding_idxs=input_binding_idxs,
            output_binding_idxs=output_binding_idxs,
            stream=stream,
        )

print_timings(name="TensorRT (INT-8)", timings=time_buffer)
del engine, context

[TensorRT (INT-8)] mean=15.52ms, sd=0.62ms, min=14.49ms, max=17.17ms, median=15.15ms, 95p=16.67ms, 99p=17.03ms


## Pytorch baseline

### Finetuning

In [ ]:
model_roberta: PreTrainedModel = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=num_labels
)
model_roberta = model_roberta.cuda()

args.learning_rate = 1e-5

trainer = Trainer(
    model_roberta,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
transformers.logging.set_verbosity_error()
trainer.train()
print(trainer.evaluate())
# {'eval_loss': 0.3559744358062744, 'eval_accuracy': 0.8655119714722364, 'eval_runtime': 19.6678, 'eval_samples_per_second': 499.04, 'eval_steps_per_second': 7.83, 'epoch': 0.98}
trainer.save_model("roberta-baseline")
del model_roberta
del trainer

[INFO|trainer.py:437] 2021-12-09 23:38:27,822 >> Using amp half precision backend


{'loss': 0.6612, 'learning_rate': 9.1875814863103e-06, 'epoch': 0.08}
{'eval_loss': 0.4690713882446289, 'eval_accuracy': 0.8217014773306164, 'eval_runtime': 19.0461, 'eval_samples_per_second': 515.328, 'eval_steps_per_second': 8.086, 'epoch': 0.08}
{'loss': 0.4984, 'learning_rate': 8.372718383311604e-06, 'epoch': 0.16}
{'eval_loss': 0.4219346344470978, 'eval_accuracy': 0.835048395313296, 'eval_runtime': 18.9872, 'eval_samples_per_second': 516.928, 'eval_steps_per_second': 8.111, 'epoch': 0.16}
{'loss': 0.4664, 'learning_rate': 7.558670143415907e-06, 'epoch': 0.24}
{'eval_loss': 0.4248501658439636, 'eval_accuracy': 0.835048395313296, 'eval_runtime': 18.5315, 'eval_samples_per_second': 529.639, 'eval_steps_per_second': 8.31, 'epoch': 0.24}
{'loss': 0.4471, 'learning_rate': 6.743807040417211e-06, 'epoch': 0.33}
{'eval_loss': 0.3851495087146759, 'eval_accuracy': 0.853998981151299, 'eval_runtime': 18.5175, 'eval_samples_per_second': 530.039, 'eval_steps_per_second': 8.316, 'epoch': 0.33}
{'

### GPU execution

To finish, we will measure vanilla Pytorch inference on both FP32 and FP16 precision, it will be our baseline:

In [ ]:
baseline_model = AutoModelForSequenceClassification.from_pretrained("roberta-baseline", num_labels=num_labels)
baseline_model = baseline_model.cuda()
baseline_model = baseline_model.eval()

data = encoded_dataset["train"][0:batch_size]
input_torch: OD[str, torch.Tensor] = convert_tensor(data=data, output="torch")

with torch.inference_mode():
    for _ in range(30):
        _ = baseline_model(**input_torch)
        torch.cuda.synchronize()
    time_buffer = list()
    for _ in range(100):
        with track_infer_time(time_buffer):
            _ = baseline_model(**input_torch)
            torch.cuda.synchronize()
print_timings(name="Pytorch (FP32)", timings=time_buffer)

[Pytorch (FP32)] mean=79.48ms, sd=0.84ms, min=78.64ms, max=82.72ms, median=79.26ms, 95p=81.66ms, 99p=82.47ms


In [ ]:
with torch.inference_mode():
    with torch.cuda.amp.autocast():
        for _ in range(30):
            _ = baseline_model(**input_torch)
            torch.cuda.synchronize()
        time_buffer = []
        for _ in range(100):
            with track_infer_time(time_buffer):
                _ = baseline_model(**input_torch)
                torch.cuda.synchronize()
print_timings(name="Pytorch (FP16)", timings=time_buffer)
del baseline_model

[Pytorch (FP16)] mean=58.02ms, sd=0.59ms, min=57.46ms, max=60.90ms, median=57.80ms, 95p=59.52ms, 99p=60.49ms


### CPU execution

In [ ]:
baseline_model = AutoModelForSequenceClassification.from_pretrained("roberta-baseline", num_labels=num_labels)
baseline_model = baseline_model.eval()
data = encoded_dataset["train"][0:batch_size]
input_torch: OD[str, torch.Tensor] = convert_tensor(data=data, output="torch")
input_torch_cpu = {k: v.to("cpu") for k, v in input_torch.items()}

import os

torch.set_num_threads(os.cpu_count())

with torch.inference_mode():
    for _ in range(3):
        _ = baseline_model(**input_torch_cpu)
        torch.cuda.synchronize()
    time_buffer = list()
    for _ in range(10):
        with track_infer_time(time_buffer):
            _ = baseline_model(**input_torch_cpu)
            torch.cuda.synchronize()
print_timings(name="Pytorch (FP32) - CPU", timings=time_buffer)

[Pytorch (FP32) - CPU] mean=3999.80ms, sd=27.86ms, min=3946.75ms, max=4042.56ms, median=4000.13ms, 95p=4037.90ms, 99p=4041.63ms


In [ ]:
with torch.inference_mode():
    with torch.cuda.amp.autocast():
        for _ in range(3):
            _ = baseline_model(**input_torch_cpu)
            torch.cuda.synchronize()
        time_buffer = []
        for _ in range(10):
            with track_infer_time(time_buffer):
                _ = baseline_model(**input_torch_cpu)
                torch.cuda.synchronize()
print_timings(name="Pytorch (FP16) - CPU", timings=time_buffer)
del baseline_model

[Pytorch (FP16) - CPU] mean=4005.57ms, sd=46.34ms, min=3922.37ms, max=4095.30ms, median=4010.20ms, 95p=4071.75ms, 99p=4090.59ms


Below, we will perform dynamic quantization on CPU.

In [ ]:
quantized_baseline_model = AutoModelForSequenceClassification.from_pretrained("roberta-baseline", num_labels=num_labels)
quantized_baseline_model = quantized_baseline_model.eval()
quantized_baseline_model = torch.quantization.quantize_dynamic(
    quantized_baseline_model, {torch.nn.Linear}, dtype=torch.qint8
)

with torch.inference_mode():
    for _ in range(3):
        _ = quantized_baseline_model(**input_torch_cpu)
        torch.cuda.synchronize()
    time_buffer = list()
    for _ in range(10):
        with track_infer_time(time_buffer):
            _ = quantized_baseline_model(**input_torch_cpu)
            torch.cuda.synchronize()
print_timings(name="Pytorch (INT-8) - CPU", timings=time_buffer)
del quantized_baseline_model

[Pytorch (INT-8) - CPU] mean=3669.84ms, sd=25.49ms, min=3633.48ms, max=3712.22ms, median=3670.07ms, 95p=3706.98ms, 99p=3711.17ms


## TensorRT baseline

Below we export a randomly initialized `Roberta` model, the purpose is to only check the performance on mixed precision (FP16, no quantization).

In [ ]:
baseline_model = AutoModelForSequenceClassification.from_pretrained("roberta-baseline", num_labels=num_labels)
baseline_model = baseline_model.cuda()
convert_to_onnx(baseline_model, output_path="baseline.onnx", inputs_pytorch=input_torch, opset=12)
del baseline_model

In [ ]:
engine = build_engine(
    runtime=runtime,
    onnx_file_path="baseline.onnx",
    logger=trt_logger,
    min_shape=(batch_size, max_seq_len),
    optimal_shape=(batch_size, max_seq_len),
    max_shape=(batch_size, max_seq_len),
    workspace_size=10000 * 1024 * 1024,
    fp16=True,
    int8=False,
)
stream: Stream = pycuda.driver.Stream()
context: IExecutionContext = engine.create_execution_context()
context.set_optimization_profile_async(profile_index=profile_index, stream_handle=stream.handle)
input_binding_idxs, output_binding_idxs = get_binding_idxs(engine, profile_index)  # type: List[int], List[int]
for _ in range(30):
    _ = infer_tensorrt(
        context=context,
        host_inputs=input_np,
        input_binding_idxs=input_binding_idxs,
        output_binding_idxs=output_binding_idxs,
        stream=stream,
    )
time_buffer = list()
for _ in range(100):
    with track_infer_time(time_buffer):
        _ = infer_tensorrt(
            context=context,
            host_inputs=input_np,
            input_binding_idxs=input_binding_idxs,
            output_binding_idxs=output_binding_idxs,
            stream=stream,
        )

print_timings(name="TensorRT (FP16)", timings=time_buffer)
del engine, context

[TensorRT (FP16)] mean=30.23ms, sd=0.49ms, min=29.73ms, max=32.67ms, median=30.06ms, 95p=30.82ms, 99p=32.38ms


## ONNX Runtime baseline

ONNX Runtime is the go to inference solution from Microsoft.

The recent 1.10 version of ONNX Runtime (with TensorRT support) is still a bit buggy on transformer models, that is why we use the 1.9.0 version in the measures below.

As before, CPU quantization is dynamic.
Function `create_model_for_provider` will set ONNX Runtime to use all cores available and enable any possible optimizations.

In [ ]:
from transformer_deploy.backends.ort_utils import optimize_onnx, create_model_for_provider
from onnxruntime.quantization import quantize_dynamic, QuantType

optimize_onnx(
    onnx_path="baseline.onnx",
    onnx_optim_fp16_path="baseline-optimized.onnx",
    use_cuda=True,
)
onnx_model = create_model_for_provider(path="baseline-optimized.onnx", provider_to_use="CUDAExecutionProvider")
quantize_dynamic("baseline-optimized.onnx", "baseline-quantized.onnx", weight_type=QuantType.QUInt8)

In [ ]:
labels = [item["label"] for item in encoded_dataset[validation_key]]
data = encoded_dataset[validation_key][0:batch_size]
inputs_onnx: OD[str, np.ndarray] = convert_tensor(data=data, output="np")
for k, v in inputs_onnx.items():
    inputs_onnx[k] = v.astype(np.int64)

model = create_model_for_provider(path="baseline-optimized.onnx", provider_to_use="CUDAExecutionProvider")
output = model.run(None, inputs_onnx)

In [ ]:
data = encoded_dataset["train"][0:batch_size]
inputs_onnx: OD[str, np.ndarray] = convert_tensor(data=data, output="np")
for k, v in inputs_onnx.items():
    inputs_onnx[k] = v.astype(np.int64)

for provider, model_path, benchmark_name, warmup, nb_inference in [
    ("CUDAExecutionProvider", "baseline.onnx", "ONNX Runtime GPU (FP32)", 10, 100),
    ("CUDAExecutionProvider", "baseline-optimized.onnx", "ONNX Runtime GPU (FP16)", 10, 100),
    ("CPUExecutionProvider", "baseline.onnx", "ONNX Runtime CPU (FP32)", 3, 10),
    ("CPUExecutionProvider", "baseline-optimized.onnx", "ONNX Runtime CPU (FP16)", 3, 10),
    ("CPUExecutionProvider", "baseline-quantized.onnx", "ONNX Runtime CPU (INT-8)", 3, 10),
]:
    model = create_model_for_provider(path=model_path, provider_to_use=provider)
    for _ in range(warmup):
        _ = model.run(None, inputs_onnx)
    time_buffer = []
    for _ in range(nb_inference):
        with track_infer_time(time_buffer):
            _ = model.run(None, inputs_onnx)
    print_timings(name=benchmark_name, timings=time_buffer)
    del model

[ONNX Runtime GPU (FP32)] mean=74.48ms, sd=0.55ms, min=73.87ms, max=76.61ms, median=74.36ms, 95p=75.89ms, 99p=76.34ms
[ONNX Runtime GPU (FP16)] mean=33.50ms, sd=0.62ms, min=32.90ms, max=37.58ms, median=33.39ms, 95p=34.42ms, 99p=35.47ms
[ONNX Runtime CPU (FP32)] mean=3767.02ms, sd=32.02ms, min=3720.72ms, max=3831.88ms, median=3766.35ms, 95p=3816.04ms, 99p=3828.71ms
[ONNX Runtime CPU (FP16)] mean=4607.67ms, sd=121.41ms, min=4513.24ms, max=4950.20ms, median=4573.18ms, 95p=4822.23ms, 99p=4924.61ms
[ONNX Runtime CPU (INT-8)] mean=3712.67ms, sd=45.19ms, min=3656.30ms, max=3827.99ms, median=3709.21ms, 95p=3788.00ms, 99p=3819.99ms


Measure of the accuracy with ONNX Runtime engine and CUDA provider:

In [ ]:
model = create_model_for_provider(path="baseline.onnx", provider_to_use="CUDAExecutionProvider")
infer_ort = lambda tokens: model.run(None, tokens)
measure_accuracy(infer=infer_ort, int64=True)

  0%|          | 0/307 [00:00<?, ?it/s]

0.8678553234844626

In [ ]:
model = create_model_for_provider(path="baseline-optimized.onnx", provider_to_use="CUDAExecutionProvider")
infer_ort = lambda tokens: model.run(None, tokens)
measure_accuracy(infer=infer_ort, int64=True)

  0%|          | 0/307 [00:00<?, ?it/s]

0.8675496688741722